# Kepler-ECG: Pipeline

## FASE 0: Setup & Domain Understanding

### Objective

Configure environment, download data, understand domain.

### Step 0.1: Create project structure

```bash
kepler-ecg/
├── data/
│ └── raw/              # Original ECG
├── src/kepler_ecg/
│ ├── preprocessing/    # Filters, quality, beat segmentation
│ ├── features/         # Features Extractors
│ └── discovery/        # Symbolic regression
├── scripts/            # Executable scripts
├── results/            # Output analysis
└── test/               # Unit tests
```

---

### Step 0.2: Install dependencies

```bash
# Core Data Science stack: math, signal tools, dataframes, and visualization
pip install numpy scipy pandas matplotlib seaborn

# Specialized Biomedical Signal Processing (essential for PhysioNet datasets)
# wfdb: read/write PhysioNet files
# neurokit2/biosppy: advanced ECG/EEG feature extraction and cleaning
pip install wfdb neurokit2 biosppy

# Symbolic Regression: Discovering mathematical expressions from data
pip install pysr

# Utilities and Performance:
# tqdm: progress bars for loops
# pyarrow: high-performance data storage and retrieval
# pytest: framework for testing your preprocessing pipeline
pip install tqdm pyarrow pytest
```

---

### Step 0.3: Multi-Dataset Registry

**Script**: [src/core/dataset_registry.py](../src/core/dataset_registry.py)

The **Dataset Registry** is the central configuration engine that enables **Kepler-ECG** to support multiple heart-signal datasets (such as PTB-XL, Chapman, Georgia, and MIT-BIH) through a unified interface. It acts as a "source of truth" for signal properties, defining the sampling rates, lead configurations, and file formats (WFDB, MAT, CSV) for each source. By abstracting the complexities of different metadata structures and labeling schemas (SCP-Codes, SNOMED, etc.), it allows the rest of the pipeline to remain dataset-agnostic, facilitating large-scale cross-database research.

#### Input

The registry is primarily accessed via its internal configuration classes and utility functions:

* **`dataset_name` (str)**: A unique identifier for the requested dataset (e.g., `'ptb-xl'`, `'mit-bih'`, or `'chapman'`).
* **`data_path` (Path/str)**: The local system path to the raw dataset files.
* **`_registry` (Internal)**: A singleton object containing predefined **`DatasetConfig`** dataclasses for every supported source.
* **`auto_detect` (Utility)**: Can take a directory path and scan for specific metadata files (like `ptbxl_database.csv`) to identify the dataset automatically.

#### Output

The module provides structured configuration objects and metadata mapping:

* **`DatasetConfig` Object**: A comprehensive container including:
* **Signal Metadata**: Sampling rate (Hz), number of leads, and lead names.
* **Format Specifiers**: The `FileFormat` enum (e.g., `WFDB`) and `LabelSource` enum.
* **Processing Functions**: References to specific loader functions for that dataset's unique metadata structure.


* **Registry Summary**: A descriptive string listing all available datasets and their basic parameters for logging purposes.
* **Detection Result**: The identified dataset name string if the auto-detection utility is used.

---

### Step 0.4: Download dataset

**Script**: [scripts/download_dataset.py](../scripts/download_dataset.py)

Python utility designed to automate the download of clinical datasets from the **PhysioNet** platform. The script acts as a wrapper around the 'wget' tool, handling automatic directory creation and file organization mirroring the source, eliminating redundant server folder hierarchies (via the 'cut-dirs' option). It is optimized to be integrated into cross-platform search pipelines.

#### Input (Commandline Arguments)

The file accepts the following parameters via 'argparse':

| Parameter | Type | Mandatory | Description |
|-----------|------|-----------|-------------|
| 'url' | String | **Required** | The direct URL of the dataset on PhysioNet (e.g. 'https://physionet.org/files/ptb-xl/1.0.3/`). |
| '--output' | Path (Path) | Optional | The local destination folder. Default: 'data/raw'. |

#### Output

* **Local Files:** A mirror copy of the downloaded dataset in the specified folder (keeping the original structure of the '.dat', '.hea', '.csv', etc. files).
* **Console:** Real-time log of download status and final confirmation of save path.
* **Exit Status:** Returns '0' on success, '1' on error (e.g. 'wget' not installed or URL unreachable).


#### Running the script for different datasets (e.g. ptb-xl)

```bash
# Download ptb-xl:
python scripts/download_dataset.py --dataset ptb-xl

# List of avaiable dataset:
python scripts/download_dataset.py --list

# Download all dataset:
python scripts/download_dataset.py --all
```

**Output**: 
- Folders containing data and metadata

---
---

## FASE 1: Preprocessing Pipeline

### Objective

Create robust pipeline: filters, quality assessment, R peak detection.

### Step 1.1: BaselineRemover: High-pass Butterworth 0.5Hz & NoiseFilter: low-pass 40Hz + notch 50Hz

**Script**: [src/preprocessing/filters.py](../src/preprocessing/filters.py)

A specialized Python module for ECG signal preprocessing that implements high-performance filtering techniques. It provides tools to eliminate common cardiac signal artifacts, such as low-frequency baseline wander (caused by breathing or movement) and high-frequency noise (EMG interference or powerline hum). The module utilizes zero-phase filtering (`filtfilt`) to ensure that the morphological timing of the ECG waves (P, QRS, and T) remains undistorted, which is critical for accurate deep learning analysis.

#### Input

The classes (`BaselineRemover`, `NoiseFilter`, `ECGFilter`) and convenience functions accept the following primary inputs:

* **`signal` (np.ndarray):** The raw ECG data. It can be 1D (a single lead) or 2D (multiple leads, shaped as `n_samples, n_leads`).
* **`fs` (int):** The sampling frequency of the signal in Hz (e.g., 500 Hz for PTB-XL).
* **Configuration Parameters:**
* **Baseline Removal:** `baseline_cutoff` (default 0.5 Hz) and `baseline_order`.
* **Noise Filtering:** `lowpass_cutoff` (default 40.0 Hz) and `lowpass_order`.
* **Powerline Interference:** `notch_freq` (50 Hz for Europe/Italy or 60 Hz) and `notch_q` (quality factor).

#### Output

* **Filtered Signal (np.ndarray):** A NumPy array of the same shape as the input, containing the cleaned ECG signal with artifacts removed.
* **Frequency Response (Optional):** Magnitude response data in dB for visualizing filter performance.
* **Metadata:** String representations (`__repr__`) of the filter objects for logging experiment configurations.

---

### Step 1.2: QualityAssessor: SNR, flatlines, saturation

**Script**: [src/preprocessing/quality.py](../src/preprocessing/quality.py)

This module provides a comprehensive framework for evaluating the clinical and technical quality of ECG signals. It implements a multi-metric approach to detect common signal acquisition issues such as **electrode disconnection (flatlines)**, **amplifier saturation (clipping)**, **baseline instability**, and **excessive noise**. By aggregating these metrics into a weighted **Quality Score**, the module allows automated pipelines to decide whether a signal is reliable enough for deep learning inference or if it should be rejected to avoid false positives.


#### Input

The module's main components (`QualityAssessor`, `assess_quality`) take the following inputs:

* **`signal` (np.ndarray):** The ECG data to be evaluated. It supports **1D arrays** (single lead) or **2D arrays** (multi-lead, e.g., 12-lead ECGs).
* **`fs` (int):** The sampling frequency in Hz, used to define temporal windows for flatline detection and frequency bands for SNR calculation.
* **`config` (QualityConfig, optional):** A custom configuration object to override default thresholds for:
* **SNR limits** (default min: 5.0 dB).
* **Flatline duration** (default min: 0.1s).
* **Amplitude ranges** (default: 0.1mV to 5.0mV).

#### Output

The module returns a **`QualityMetrics`** object containing:

* **Numerical Metrics:** Specific values for `snr_db`, `flatline_ratio`, `saturation_ratio`, `baseline_drift`, and `high_freq_noise`.
* **`quality_score` (float):** A normalized value from **0.0 to 1.0** representing overall signal integrity.
* **`quality_level` (Enum):** A categorical label: `EXCELLENT`, `GOOD`, `ACCEPTABLE`, `POOR`, or `UNUSABLE`.
* **`is_usable` (bool):** A boolean flag indicating if the signal meets the minimum quality threshold for analysis.
* **`issues` (List[str]):** A detailed list of detected problems (e.g., "Lead 0: Low SNR", "Signal saturation").

---

### Step 1.3: BeatSegmenter: Pan-Tompkins R-detection

**Script**: [src/preprocessing/segmentation.py](../src/preprocessing/segmentation.py)

This module is the core engine for signal parsing within the **Kepler-ECG** project. It implements the classic **Pan-Tompkins algorithm** to detect R-peaks (the most prominent spikes in an ECG) with high precision. Beyond simple detection, it automatically segments the continuous cardiac signal into individual heartbeats, aligns them, and computes vital clinical metrics such as Heart Rate (BPM) and RR intervals. This segmentation is a fundamental prerequisite for training deep learning models on heartbeat-level morphologies.

#### Input

The primary classes (`PanTompkinsDetector`, `BeatSegmenter`) and the `segment_beats` function accept the following:

* **`signal` (np.ndarray):** The raw or filtered ECG data. It supports **1D arrays** (single lead) or **2D arrays** (multi-lead).
* **`fs` (int):** The sampling frequency in Hz (essential for time-to-sample conversions).
* **`config` (SegmentationConfig, optional):** A configuration object to tune the algorithm, including:
* **Bandpass limits:** Default 5.0 Hz to 15.0 Hz to isolate QRS energy.
* **Search windows:** Time before (0.2s) and after (0.4s) the R-peak for beat extraction.
* **Physiological limits:** Min/Max RR intervals to filter out noise that mimics heartbeats.

#### Output

The module returns a **`SegmentationResult`** object containing:

* **`r_peaks` (np.ndarray):** The precise indices of all detected heartbeats in the signal.
* **`beats` (np.ndarray):** A 2D matrix of shape `(n_beats, beat_length)` containing the extracted and aligned heartbeat waveforms.
* **`heart_rate_bpm` (float):** The calculated mean heart rate.
* **`detection_confidence` (float):** A score from **0.0 to 1.0** indicating the reliability of the detection based on rhythm regularity and signal consistency.
* **`beat_template` (np.ndarray):** An "average" heartbeat calculated from all valid segments, useful for noise reduction.

---

### Step 1.4: HRVPreprocessor: rimozione ectopici, interpolazione

**Script**: [src/preprocessing/hrv_preprocessing.py](../src/preprocessing/hrv_preprocessing.py)

This module is dedicated to the specific preprocessing required for **Heart Rate Variability (HRV)** analysis. Its primary purpose is to transform a sequence of raw RR intervals into a clean, uniform time series ready for time and frequency domain analysis. The module implements advanced algorithms for the identification of **ectopic beats** (irregular rhythms) and artifacts, utilizing clinical standards such as the Malik or Kamath criteria. Additionally, it includes interpolation functionality to handle missing data and ensure a constant sampling rate, which is essential for accurate spectral analysis of cardiac variability.

#### Input

The primary components of the module, such as the `HRVPreprocessor` class and the `preprocess_hrv` function, require the following inputs:

* **`rr_intervals` (np.ndarray)**: An array containing RR intervals expressed in milliseconds.
* **`config` (HRVConfig, optional)**: A configuration object used to customize the processing pipeline. This includes:
* **Ectopic detection method**: Options include `MALIK`, `KAMATH`, `KARLSSON`, or `ACAR`.
* **Detection threshold**: A numerical parameter to adjust the sensitivity of the artifact identification algorithm.
* **Resampling frequency**: The target frequency (e.g., 4 Hz) for the interpolation of the time series.



#### 📤 Output

The module returns an **`HRVData`** object containing the following elements:

* **`rr_clean` (np.ndarray)**: The series of RR intervals after the removal or correction of identified ectopic beats.
* **`rr_interpolated` (np.ndarray)**: An equidistantly sampled time series, suitable for frequency-based analysis such as Fast Fourier Transform (FFT).
* **`ectopic_mask` (np.ndarray)**: A boolean array indicating which original beats were flagged as anomalies or artifacts.
* **`metrics` (Dict)**: Basic descriptive statistics, such as the percentage of detected ectopic beats within the signal.

---

### Step 1.5: PreprocessingPipeline

**Script**: [src/preprocessing/pipeline.py](../src/preprocessing/pipeline.py)

This module serves as the central orchestration engine for the **Kepler-ECG** project, integrating all individual preprocessing components into a unified and automated workflow. It sequentially executes signal filtering (baseline and noise removal), quality assessment, R-peak detection, beat segmentation, and HRV analysis. Designed for high-performance research, the pipeline features **parallel processing** to handle large datasets efficiently and a **caching system** (using SHA-256 hashing) to avoid redundant computations. It provides a robust interface for transforming raw physiological data into structured, analysis-ready objects.

#### Input

The `PreprocessingPipeline` class and its convenience functions (like `process_ecg_batch`) accept the following:

* **`signal` (np.ndarray)**: The raw ECG waveform, supporting both single-lead (1D) and multi-lead (2D) configurations.
* **`fs` (int)**: The sampling frequency in Hz, required by all internal stages for accurate time-domain processing.
* **`PipelineConfig` (optional)**: A comprehensive configuration object that allows fine-tuning of:
* **Filter Settings**: Cutoff frequencies for high-pass, low-pass, and notch filters.
* **Execution Settings**: `n_jobs` for parallel execution and `enable_cache` to toggle disk-based storage of results.
* **Quality & Segmentation**: Thresholds for signal rejection and beat extraction parameters.

#### Output

The pipeline returns a **`ProcessedECG`** object (or a list of objects for batches), which contains:

* **`filtered_signal`**: The cleaned ECG data.
* **`quality_metrics`**: The results from the quality assessment stage (score, level, and usability).
* **`segmentation`**: R-peak indices, extracted heartbeat segments, and heart rate statistics.
* **`hrv_data`**: Cleaned and interpolated RR intervals for variability analysis.
* **`metadata`**: A dictionary containing processing time, timestamps, and the unique hash identifier for the operation.

---

### Step 1.6: Processare intero dataset

**Script**: [scripts/process_dataset.py](../scripts/process_dataset.py)

This module is a high-level CLI (Command Line Interface) script designed to automate the processing of large-scale ECG datasets. It acts as a bridge between raw data storage and the **Kepler-ECG Preprocessing Pipeline**, featuring an auto-detection system that recognizes various clinical data formats such as **WFDB** (.dat/.hea), **MAT** (.mat), and **NumPy** (.npy). The script is optimized for research workflows, allowing for targeted sampling rate filtering and batch processing to handle thousands of records while providing detailed logging and progress reporting.

#### Input (Command Line Arguments)

The script requires several parameters to define the data workflow:

* **`--data_dir` (Required)**: The path to the directory containing the raw ECG dataset.
* **`--output_dir` (Required)**: The path where the processed results, logs, and metadata will be saved.
* **`--sampling_rate` (Optional)**: A filter to process only files with a specific sampling frequency (e.g., 500 Hz for PTB-XL).
* **`--n_samples` (Optional)**: Limits the processing to a specific number of records (useful for testing).
* **`--batch_size` (Default: 100)**: Determines how many records are processed before reporting progress.

#### Output

The execution produces a structured output in the destination folder:

* **Processed Data**: Individual preprocessed objects (typically saved as `.pkl` or `.npy`) containing filtered signals and extracted features.
* **`processing_summary.json`**: A global metadata file summarizing the execution time, number of files processed, and any errors encountered.
* **Quality Statistics**: A detailed breakdown in the logs showing the distribution of signal quality levels (e.g., % of Excellent vs. Unusable signals).
* **Execution Logs**: A professional log file tracking every step of the batch operation.

#### Running the script for different datasets

```bash
# 1. Help
python scripts/process_dataset.py --help

# 2. Process ptb-xl
python scripts/process_dataset.py --dataset ptb-xl --sampling_rate 500
```

---
---

## 📊 FASE 2: Features Extraction

### Objective

Extract 100+ features for ECG: morphological, spectral, wavelet, compressibility.

### Step 2.1: MorphologicalExtractor

**Script**: [src/features/morphological.py](../src/features/morphological.py)

This module is designed to extract detailed clinical and morphological features from ECG beat templates. It identifies key characteristic points of the cardiac cycle, such as the onset, peak, and offset of P, Q, R, S, and T waves. By calculating specific wave durations, amplitudes, and intervals (including corrected QT intervals using Bazett, Fridericia, and Framingham formulas), the module provides a structured digital representation of the heartbeat's shape. These features are essential for both traditional clinical diagnostics and as structured inputs for deep learning classifiers.

#### Input

The core components, specifically the `MorphologicalExtractor` and the `WavePoints` data structure, process the following inputs:

* **`beat_template` (np.ndarray)**: A 1D array representing a single, representative heartbeat (typically the average or median beat) centered on the R-peak.
* **`fs` (int)**: The sampling frequency in Hz, used to convert sample indices into clinical time measurements in milliseconds.
* **`rr_mean_ms` (optional float)**: The mean RR interval in milliseconds, required for calculating corrected QT (QTc) intervals.

#### Output

The module returns a comprehensive dictionary or data structure containing:

* **Wave Amplitudes**: Peak values for P, Q, R, S, and T waves, along with ST-segment levels.
* **Temporal Intervals**: Precise durations in milliseconds for the P-wave, QRS complex, T-wave, PR interval, and QT interval.
* **Corrected Metrics**: QTc values calculated via various medical formulas (Bazett, Fridericia, Framingham).
* **Morphological Ratios**: Calculated values such as the R/S ratio and T/R ratio, as well as areas under the QRS and T curves.

---

### Step 2.2: SpectralAnalyzer (HRV)

**Script**: [src/features/spectral.py](../src/features/spectral.py)

This module focuses on the frequency-domain analysis of Heart Rate Variability (HRV). It transforms clean RR interval sequences into power spectral density (PSD) estimates to quantify the autonomic nervous system's influence on the heart. The module implements both the **Welch periodogram** for uniformly resampled series and the **Lomb-Scargle periodogram** for non-uniform data. It is designed to extract standard clinical frequency bands (VLF, LF, HF) and calculate the LF/HF ratio, which serves as an indicator of the sympathetic-vagal balance.

#### Input

The primary class `SpectralAnalyzer` and its associated functions take the following inputs:

* **`rr_intervals` (np.ndarray)**: An array of RR intervals in milliseconds.
* **`fs_interpolation` (float, default=4.0)**: The frequency in Hz used to resample the RR series for Welch analysis.
* **`method` (Literal['welch', 'lomb-scargle'])**: The spectral estimation technique to be applied.
* **`bands` (FrequencyBands, optional)**: A configuration object defining the frequency limits for:
* **VLF (Very Low Frequency)**: 0.003 - 0.04 Hz.
* **LF (Low Frequency)**: 0.04 - 0.15 Hz.
* **HF (High Frequency)**: 0.15 - 0.4 Hz.

#### Output

The module returns a **`SpectralMetrics`** object (or a dictionary) containing:

* **Absolute Power**: Total power and power for each specific band (VLF, LF, HF) measured in .
* **Normalized Power**: LF and HF components expressed in normalized units (n.u.).
* **LF/HF Ratio**: A numerical value representing the balance between the sympathetic and parasympathetic systems.
* **Peak Frequencies**: The specific frequency values (in Hz) where the maximum power occurs within each band.
* **PSD Data**: The frequency vector and power spectral density values for visualization (e.g., plotting the periodogram).

---

### Step 2.3: WaveletExtractor

**Script**: [src/features/wavelet.py](../src/features/wavelet.py)

This module implements feature extraction based on the **Discrete Wavelet Transform (DWT)**, a fundamental technique for the time-frequency analysis of ECG signals. By decomposing the signal into different scales (levels), the module isolates fine details like spikes and noise at lower scales, the QRS complex at middle scales, and P/T waves or baseline wander at higher scales. It is particularly effective at capturing non-stationary morphological variations that traditional methods might miss, providing a set of mathematical descriptors (energy, entropy, and statistical parameters) ideal for Deep Learning models.

#### Input

The `WaveletExtractor` class and its primary functions accept the following:

* **`signal` (np.ndarray)**: The ECG signal (either a segmented beat or a continuous strip) to be decomposed.
* **`WaveletConfig` (optional)**: A configuration object used to define:
* **`wavelet`**: The type of mother wavelet (defaults to `'db4'` — Daubechies 4, which is highly effective for ECG morphology).
* **`max_level`**: The number of decomposition levels (if `None`, it is automatically calculated based on signal length).
* **`mode`**: The signal extension mode (e.g., `'symmetric'`).

#### Output

The module returns a dictionary or a **`WaveletFeatures`** object containing:

* **`energy_per_level`**: The distribution of signal energy across the various approximation and detail levels.
* **`wavelet_entropy`**: A measure of signal complexity or disorder based on the wavelet energy distribution.
* **`coefficient_statistics`**: A set of statistical parameters (mean, standard deviation, skewness, and kurtosis) calculated for each decomposition level.
* **`coefficients`**: The raw wavelet coefficients (optional), useful for signal reconstruction or custom filtering.

---

### Step 2.4: CompressibilityCalculator ⭐ CORE

**Script**: [src/features/compressor.py](../src/features/compressor.py)

This module implements advanced techniques to measure the **algorithmic complexity** and regularity of ECG signals and RR time series. It utilizes compressibility as a proxy for Kolmogorov complexity, based on the premise that more regular and predictable signals are more easily compressed. The module includes classic entropy algorithms (Sample, Approximate, Permutation) and Lempel-Ziv complexity metrics, providing a non-linear characterization of the signal that complements standard morphological and spectral analyses.

#### Input

The primary components, such as the `CompressibilityCalculator` class, accept the following parameters:

* **`signal` (np.ndarray)**: The filtered ECG signal or the RR interval series.
* **`CompressibilityConfig` (optional)**: A configuration object to tune the algorithm parameters:
* **`entropy_m`**: Embedding dimension for entropy calculation (default: 2).
* **`entropy_r_factor`**: Tolerance factor (default: 0.2 * standard deviation of the signal).
* **`perm_order` & `perm_delay**`: Specific parameters for Permutation Entropy.

#### Output

The module returns a dictionary of numerical features (**`Dict[str, float]`**) including:

* **Compression Ratios**: Ratios calculated using `gzip`, `bzip2`, and `lzma` algorithms.
* **Entropy Metrics**: Values for *Sample Entropy*, *Approximate Entropy*, *Permutation Entropy*, and *Shannon Entropy*.
* **Complexity Measures**: Lempel-Ziv complexity index and Kolmogorov complexity estimates.
* **Hjorth Parameters**: Statistical time-domain metrics for activity, mobility, and complexity.

---

### Step 2.5: Unified Label Schema Module

**Script**: [src/core/label_schema.py](../src/core/label_schema.py)

The **Unified Label Schema** is the taxonomic heart of the **Kepler-ECG** multi-dataset framework. It solves the critical challenge of clinical "interoperability" by mapping disparate diagnostic codes from various international databases (like SCP-ECG codes from PTB-XL or SNOMED-CT from the Chapman dataset) into a standardized, hierarchical structure. By organizing cardiac conditions into five major superclasses—**NORM, MI, STTC, CD, and HYP**—it allows models trained on one dataset to be validated against another, ensuring a consistent ground truth for both Deep Learning and Symbolic Regression tasks.

#### Input

The `LabelMapper` class and its primary methods (e.g., `map_scp_codes`, `map_snomed`) accept:

* **`codes` (Dict or List)**: The raw diagnostic output from a dataset. This could be a dictionary of SCP-ECG codes with confidence scores or a list of SNOMED-CT numerical strings.
* **`dataset_name` (str)**: Used by the mapper to select the appropriate internal translation table (e.g., `'ptb-xl'`, `'chapman'`, `'georgia'`).
* **`threshold` (float, default=0.0)**: A confidence filter to ignore diagnostic labels that fall below a certain probability in the original metadata.

#### Output

The module returns structured **`Label`** objects and vectors:

* **`mapped_labels` (List[Label])**: A list of objects containing the `Superclass`, `Subclass`, and the original `specific_code`.
* **`superclass_vector` (np.ndarray)**: A binary multi-hot encoded vector (length 5) representing the presence of NORM, MI, STTC, CD, and HYP.
* **`primary_superclass` (Enum)**: The dominant clinical category for the record, determined by the highest confidence score or clinical priority.
* **`original_codes` (List[str])**: A clean list of the original strings used for the mapping, preserved for auditing and clinical verification.

---

### Step 2.6: Feature Pipeline

**Script**: [src/features/feature_pipeline.py](../src/features/feature_pipeline.py)

The **Feature Pipeline** is the high-level orchestrator designed to consolidate all extraction methodologies within the **Kepler-ECG** ecosystem. It acts as a unified interface that coordinates the simultaneous extraction of morphological, interval-based, spectral (HRV), and wavelet-based features. By transforming preprocessed ECG signals and detected R-peaks into a comprehensive numerical feature vector, this module prepares the data for statistical analysis or as input for machine learning models. It supports batch processing with multi-threading and includes robust error handling to ensure pipeline stability during large-scale dataset operations.

#### Input

The `FeaturePipeline` class and its main execution methods (e.g., `extract_all_features`) require:

* **`signal` (np.ndarray)**: The preprocessed/filtered ECG waveform.
* **`r_peaks` (np.ndarray)**: The indices of detected R-peaks (obtained from the `Segmentation` module).
* **`sampling_rate` (int)**: The frequency (Hz) used for time-to-sample conversions.
* **`FeatureConfig` (optional)**: A structured configuration object to toggle specific extractors:
* `extract_morphological`: Boolean to enable/disable wave shape analysis.
* `extract_spectral`: Boolean for HRV frequency domain analysis.
* `extract_wavelet`: Boolean for multi-scale decomposition.
* `n_jobs`: Number of threads for parallel batch extraction.

#### Output

The pipeline returns a **`FeatureVector`** (or a list of vectors for batch mode) containing:

* **`morphological_features`**: A set of amplitudes and areas (P, Q, R, S, T waves).
* **`interval_features`**: Clinical durations (PR, QRS, QT) and corrected QTc values.
* **`spectral_features`**: Frequency band powers (VLF, LF, HF) and the LF/HF ratio.
* **`wavelet_features`**: Energy and entropy metrics across multiple decomposition levels.
* **`processing_metadata`**: Information regarding execution time and potential warnings (e.g., if a specific feature couldn't be calculated due to signal length).

---

### Step 2.7: FeaturePipeline completa

**Script**: [scripts/extract_features_dataset.py](../scripts/extract_features_dataset.py)

This script serves as the final operational utility in the **Kepler-ECG** preprocessing suite. It is a high-level command-line tool designed to perform "Phase 2" of data preparation: extracting advanced multidimensional features from a previously processed dataset. It synchronizes raw signal data with metadata, applying morphological, spectral, wavelet, and compressibility extractors simultaneously. Additionally, it integrates the **Diagnosis Mapper** to align clinical labels, resulting in a comprehensive, "ML-ready" CSV file that can be fed directly into deep learning models or statistical analysis tools.

#### Input (Command Line Arguments)

The script requires specific paths and flags to coordinate the extraction process:

* **`--phase1` (Required)**: Path to the CSV file generated during Phase 1 (e.g., `ptb-xl_features.csv`), which contains file pointers and basic metadata.
* **`--data-dir` (Required)**: The root directory where the original raw ECG files are stored.
* **`--output` (Optional)**: Specific path for the output CSV. If omitted, it appends `_features_extracted` to the input filename.
* **`--lead` (Default: 0)**: Specifies which ECG lead to extract features from (essential for multi-lead datasets).
* **Feature Toggles**: Optional flags to disable specific extractions (e.g., `--no-wavelet`, `--no-compressibility`, `--no-diagnosis`).
* **Processing Limits**: `--n_samples` and `--start` to allow for partial dataset processing or resume capabilities.

#### Output

The script generates a final structured dataset and execution logs:

* **`*ptb-xl_features_extracted.csv`**: A dense tabular file where each row represents an ECG record and columns contain the full suite of extracted features (Amplitudes, Intervals, QTc, Spectral powers, Wavelet coefficients, Entropy, and diagnostic classes).
* **Progress Telemetry**: A real-time progress bar (via `tqdm`) showing the processing rate (samples per second) and estimated time of completion.
* **Statistical Summary**: A final console output detailing the number of successful extractions versus failures (due to signal quality or missing files).

#### Running the script for different datasets

```bash
# Extract features for PTB-XL
python scripts/extract_features_dataset.py --dataset ptb-xl
```

---
---

## 🔍 FASE 3: Discovery & Analysis

### Objective
Statistical validation, visualizations, dataset preparation for SR.

### Step 3.1: Statistical Analysis

**Script**: [src/analysis/statistical.py](../src/analysis/statistical.py)

The **Statistical Analysis Module** is the core component of "Phase 3" in the **Kepler-ECG** project. It provides a robust framework for validating the clinical relevance of extracted features across different diagnostic groups. The module implements a suite of parametric and non-parametric tests (ANOVA, Kruskal-Wallis, T-tests) to determine if features like HRV spectral power or wavelet entropy vary significantly between healthy subjects and patients with cardiac pathologies. It also calculates effect sizes (Cohen's ) and performs correlation analysis to identify the most discriminative biomarkers for subsequent Deep Learning training.

#### Input

The main analytical classes (`StatisticalAnalyzer`) and their methods accept:

* **`df` (pd.DataFrame)**: The structured feature matrix (typically the `*_features_extracted.csv` file) containing both numerical features and categorical diagnostic labels.
* **`feature` (str)**: The name of the specific column to analyze (e.g., `'hf_power'`, `'qrs_duration'`).
* **`category_col` (str)**: The column used for grouping data (e.g., `'diagnostic_superclass'`).
* **`config` (optional)**: Parameters to define the significance level (default ) and minimum group size requirements.

#### Output

The module returns structured result objects (**`ANOVAResult`**, **`PairwiseComparison`**) containing:

* **Global Statistics**: F-statistic or H-statistic and their corresponding p-values to check for overall group differences.
* **Significance Flags**: Boolean indicators (`is_significant`) and visual markers (e.g., `***` for ).
* **Pairwise Comparisons**: Detailed results from post-hoc tests (e.g., Mann-Whitney U) showing exactly which diagnostic classes differ from each other.
* **Effect Sizes**: Cohen's  or similar metrics to quantify the magnitude of the difference between groups.
* **Descriptive Stats**: Group-specific means, standard deviations, and sample counts for reporting.

---

### Step 3.2: Research Visualization

**Script**: [src/analysis/visualization.py](../src/analysis/visualization.py)

The **Visualization Module** is the graphical engine for the final phase of the **Kepler-ECG** project. It is designed to produce publication-quality charts and exploratory data analysis (EDA) plots that reveal the underlying patterns in cardiac data. The module focuses on the relationship between mathematical features (Morphological, Spectral, Wavelet, and Compressibility) and clinical diagnoses. It provides advanced tools for dimensionality reduction, correlation mapping, and group-wise distribution analysis, ensuring that the results of the Deep Learning and Statistical phases are interpretable and visually compelling.


#### Input

The `ECGVisualizer` class and its plotting methods take the following inputs:

* **`df` (pd.DataFrame)**: The processed feature matrix containing numerical extractions and categorical labels (NORM, MI, STTC, CD, HYP).
* **`features` (List[str])**: A list of specific feature names to be visualized (e.g., `['samp_en', 'hf_power', 'qrs_duration']`).
* **`target_col` (str)**: The categorical column used for grouping and coloring the data (default: `diagnostic_superclass`).
* **`VisualizationConfig` (optional)**: Parameters to customize plot aesthetics, such as color palettes (e.g., the predefined `DIAGNOSIS_COLORS`), figure sizes, and DPI settings for high-resolution export.

#### Output

The module generates high-fidelity visual outputs and saved files:

* **Statistical Plots**: Box plots and violin plots showing the distribution and variance of features across different pathologies.
* **Correlation Heatmaps**: Matrices illustrating the redundancy or independence between different ECG descriptors.
* **Manifold Projections**: 2D or 3D scatter plots using **t-SNE** or **PCA** to visualize how well the different diagnostic classes cluster in the feature space.
* **Exported Files**: High-resolution images saved in multiple formats (e.g., `.png`, `.pdf`, `.svg`) in a structured `reports/` directory.

---


### Step 3.3: Feature Selection

**Script**: [src/analysis/feature_selection.py](../src/analysis/feature_selection.py)

The **Feature Selection Module** is a critical analytical component of Phase 3, designed to identify the most discriminative physiological biomarkers from the high-dimensional feature set generated in Phase 2. It implements a hybrid selection strategy combining univariate statistical tests (ANOVA F-score, Mutual Information) with multivariate machine learning models (Random Forest, Gradient Boosting). By performing redundancy analysis and calculating importance rankings, the module filters out noisy or highly correlated features, ensuring that subsequent Symbolic Regression and Deep Learning models are efficient, interpretable, and less prone to overfitting.

#### Input

The primary class `FeatureSelector` and its selection methods (e.g., `calculate_importance`, `select_best_features`) take the following inputs:

* **`df` (pd.DataFrame)**: The input feature matrix containing all extracted ECG metrics and clinical labels.
* **`target_col` (str)**: The objective variable for selection (e.g., `'diagnostic_superclass'` for classification or `'age'` for regression).
* **`n_features` (int, default=10)**: The target number of top-performing features to retain.
* **`max_correlation` (float, default=0.85)**: The threshold for redundancy filtering; if two high-ranking features are more correlated than this value, the lower-ranked one is discarded.
* **`method` (str)**: The algorithm to use for importance calculation (e.g., `'random_forest'`, `'mutual_info'`, or `'gradient_boosting'`).

#### Output

The module returns a **`SelectionSummary`** (or a detailed dictionary) containing:

* **`suggested_features` (List[str])**: The final list of selected feature names, optimized for predictive power and low redundancy.
* **`importance_scores` (pd.DataFrame)**: A ranked table showing the raw scores (F-score, Gini importance, or MI) and the final rank for every feature analyzed.
* **`group_distribution` (Dict)**: A breakdown showing how many features were selected from each category (e.g., 3 Morphological, 4 Spectral, 3 Wavelet).
* **`redundancy_report`**: Information on which features were removed due to high multicollinearity.
* **`baseline_comparison`**: Cross-validation scores showing the performance of a model using only the selected features versus the full feature set.

---

### Step 3.4: Symbolic Regression Preparation

**Script**: [src/analysis/sr_preparation.py](../src/analysis/sr_preparation.py)

This module is a specialized data engineering component designed to bridge the gap between extracted ECG features and **Symbolic Regression (SR)** tools like PySR. Its primary purpose is to transform complex feature matrices into highly optimized, clean, and normalized datasets. It supports different machine learning tasks, including binary classification (e.g., Normal vs. Pathology), multi-class classification, and regression (e.g., predicting cardiac age). By automating data cleaning, feature scaling, and train-test splitting, it ensures that the mathematical formulas discovered by SR are both statistically sound and interpretable.


#### Input

The `SRDataPreparer` class and its associated methods take the following inputs:

* **`df` (pd.DataFrame)**: The comprehensive feature matrix generated in earlier phases (containing morphological, spectral, and wavelet features).
* **`target_col` (str)**: The dependent variable to be predicted (e.g., a specific diagnostic class or a numerical value).
* **`feature_list` (List[str], optional)**: A specific subset of features to include in the analysis to reduce dimensionality.
* **`PreparationConfig` (optional)**: Configuration for data processing, including:
* **Normalization Method**: Choice between `StandardScaler` (Z-score) or `MinMaxScaler`.
* **Task Type**: Defines the objective as `binary`, `multiclass`, or `regression`.
* **Test Split Ratio**: The percentage of data reserved for validation (e.g., 0.2).

#### Output

The module returns an **`SRDataset`** object, which contains:

* **`X` (np.ndarray)**: The cleaned and scaled feature matrix ready for the regressor.
* **`y` (np.ndarray)**: The encoded target vector (numeric labels for classification or continuous values for regression).
* **`feature_names`**: A list of strings corresponding to the columns in `X`, essential for the symbolic regressor to name variables in the discovered formulas (e.g.,  becomes `hf_power`).
* **`baseline_metrics`**: A dictionary containing performance scores from a baseline model (Logistic or Linear Regression) to provide a benchmark for the Symbolic Regression results.
* **Metadata**: Information regarding the scaling parameters used, enabling the reversal of normalization for interpretation.

---

### Step 3.5: Phase 3 pipeline

**Script**: [src/analysis/pipeline.py](../src/analysis/pipeline.py)

The **Phase 3 Pipeline Orchestrator** is the final management layer of the **Kepler-ECG** project. Its role is to automate the entire analytical workflow following feature extraction. It sequentially triggers statistical validation, generates a full suite of research visualizations, performs intelligent feature selection, and exports data specifically formatted for **Symbolic Regression**. This module ensures that the transition from raw physiological features to interpretable mathematical models is consistent, reproducible, and ready for clinical reporting.

#### Input

The `Phase3Pipeline` class and its execution command accept:

* **`data_path` (str)**: The path to the comprehensive feature CSV file (e.g., `ptb-xl_features_extracted.csv`) generated in Phase 2.
* **`output_dir` (str)**: The destination folder where all reports, plots, and datasets will be stored.
* **`PipelineConfig` (optional)**: A configuration object to customize the execution:
* **`sr_features_count`**: The number of top features to select for Symbolic Regression.
* **`skip_viz`**: A boolean flag to bypass image generation for faster processing.
* **`significance_level`**: The alpha threshold (e.g., 0.05) for statistical tests.

#### Output

The pipeline produces a structured directory of results, typically including:

* **Statistical Reports**: JSON and text files summarizing ANOVA results and p-values for all biomarkers.
* **Visualization Gallery**: A subfolder containing high-resolution PNG/PDF plots (Heatmaps, Box plots, t-SNE clusters).
* **Selected Features List**: A filtered dataset containing only the most discriminative, non-redundant features.
* **SR-Ready Datasets**: Multiple CSV files (e.g., `sr_binary_MI.csv`) formatted specifically for discovery of mathematical formulas.
* **`pipeline_results.json`**: A master metadata file containing baseline performance metrics (e.g., Logistic Regression accuracy) and processing timestamps.

---

### Step 3.6:  Multi-Dataset Analysis Orchestrator

**Script**: [scripts/analyze_dataset.py](../scripts/analyze_dataset.py)

The **Dataset Analysis Script** is the primary execution entry point for **Phase 3** of the Kepler-ECG project. It acts as a high-level manager that automates the transition from raw extracted features to scientific insights. Designed with multi-dataset support in mind, it orchestrates four critical sub-processes: statistical significance testing, publication-quality visualization, intelligent feature selection, and data formatting for Symbolic Regression. This script allows researchers to process individual datasets (like PTB-XL or Chapman) or execute a full-scale comparative analysis across all available ECG databases with a single command.

#### Input (Command Line Arguments)

The script is highly configurable via CLI flags to tailor the analysis depth:

* **`--dataset` (str)**: The name of the dataset to analyze (e.g., `ptb-xl`, `chapman`, `georgia`).
* **`--all` (flag)**: Automatically detects and analyzes every processed dataset found in the results directory.
* **`--input` (Path)**: Allows manual specification of a custom CSV file if the standard directory structure isn't used.
* **`--n_features` (int, default: 12)**: Sets the target number of optimized features to select for the subsequent Symbolic Regression phase.
* **`--skip-viz` (flag)**: Bypasses the generation of plots to speed up execution in headless or server environments.

#### Output

The script generates a structured "Analysis Package" within the `results/[dataset_name]/` directory:

* **`analysis_summary.json`**: A master metadata file containing records counts, processing timestamps, and high-level success metrics.
* **Statistical & Selection Reports**: Detailed JSON files highlighting p-values, effect sizes, and the top-ranked physiological biomarkers.
* **Visualization Gallery**: A folder of high-resolution plots (Heatmaps, Box plots, PCA clusters) if not skipped.
* **SR-Ready Files**: Subsets of the data specifically cleaned and formatted for the `PySR` discovery engine in Phase 4.
* **Console Telemetry**: A real-time log showing the progress of each analytical stage and a final "Multi-Dataset Summary" table if processing multiple sources.

#### Running the pipeline for different datasets

```bash
# Lista dataset disponibili
python scripts/analyze_dataset.py --list

# Analizza PTB-XL
python scripts/analyze_dataset.py --dataset ptb-xl

# Analizza tutti (skip visualizzazioni per velocità)
python scripts/analyze_dataset.py --all --skip-viz

# Analizza Chapman con 20 features
python scripts/analyze_dataset.py --dataset chapman --n-features 20
```

---
---

## 🧬 FASE 4: Symbolic Regression Discovery, Wave Delineation & QTc Validation

### Objective
Discover formulas that can be interpreted with PySR, extract QT/RR from all ECGs, validate Kepler formulas.

### Step 4.1: Setup PySR/Julia

```bash
pip install pysr
python -c "import pysr; pysr.install()"  # Installa Julia backend
```

---

### Step 4.2: Stream A - Symbolic Regression for Diagnostic Categories Binary Classification

**Script**: [scripts/sr_classification.py](../scripts/sr_classification.py)

The **Symbolic Regression Classification Script** is an advanced discovery tool within the Kepler-ECG framework. Its primary objective is to find interpretable mathematical formulas that distinguish between specific diagnostic categories (e.g., Healthy/NORM vs. Myocardial Infarction/MI). Leveraging the `PySR` library, it evolves equations that serve as transparent alternatives to "black-box" machine learning models. The script features high-performance configuration, including logistic loss functions specifically for classification, automatic threshold calibration using Youden's J statistic, and 5-fold cross-validation to ensure the discovered formulas are medically robust and generalizable.

#### Input (Command Line Arguments)

The script can either use a pre-prepared subset or generate one from a full feature file:

* **`--input` (Path)**: Path to a specific CSV file already formatted for SR (e.g., `ptb-xl_norm_vs_hyp.csv`).
* **`--dataset` (str)**: Name of the dataset to process (e.g., `ptb-xl`, `chapman`) if using the auto-preparation mode.
* **`--positive-class` (str)**: The diagnostic category to detect (e.g., `MI`, `HYP`, `CD`).
* **`--negative-class` (default: 'NORM')**: The reference category to compare against.
* **`--all` (flag)**: If set, the script iterates through all available pathological classes in the dataset.
* **`--iterations` (int, default: 100)**: Number of evolutionary cycles for the PySR engine.

#### Output

The script generates a results directory containing the mathematical and statistical findings:

* **`sr_results.json`**: A master file containing the "Best Equation" (selected by the highest score/complexity ratio), its performance metrics, and the optimal classification threshold.
* **`hall_of_fame.csv`**: A list of all candidate equations found during the search, ranked by their Pareto efficiency (accuracy vs. simplicity).
* **Performance Benchmarks**:
* **AUC (Area Under Curve)**: Comparing the SR formula against a baseline Logistic Regression.
* **Sensitivity & Specificity**: Calculated at the optimal decision point.


* **Visualizations**:
* **ROC & Precision-Recall Curves**: Graphical performance comparison.
* **Pareto Front Plot**: Shows the trade-off between equation length and error.

#### Running

```bash
python scripts/sr_classification.py --dataset ptb-xl --positive-class HYP
python scripts/sr_classification.py --dataset ptb-xl --all  # Tutte le patologie
```
---

# 📊 Analisi Risultati SR Classification

## Sommario Esecutivo

Ho analizzato 18 classificatori SR (5 patologie × 4 dataset, meno alcuni non disponibili). Ecco i risultati organizzati per patologia.

---

## 🫀 **CD (Conduction Disturbances)** - Disturbi di Conduzione

| Dataset | AUC SR | AUC Baseline | Formula | Complessità |
|---------|--------|--------------|---------|-------------|
| **Chapman** | **0.946** | 0.970 | `0.75 * log(rr_std_ms + heart_rate_bpm + 2.70)` | 8 |
| CPSC-2018 | 0.902 | 0.922 | `log(age + rr_std_ms + 3.60) * 0.61` | 8 |
| PTB-XL | 0.789 | 0.837 | `((abs(rr_std_ms) + heart_rate_bpm) - snr_db) * 0.15` | 10 |
| Georgia | 0.822 | 0.901 | `(n_beats + age) * 0.19 + 0.5` | 7 |

**🔑 Pattern comune**: `log(rr_std_ms + heart_rate_bpm)` o combinazioni di **variabilità RR + frequenza cardiaca**

**Interpretazione**: I disturbi di conduzione alterano la regolarità del ritmo cardiaco. La formula cattura questo attraverso:
- `rr_std_ms`: variabilità degli intervalli RR (alta nei blocchi)
- `heart_rate_bpm`: frequenza (spesso alterata)
- `age`: fattore di rischio noto

---

## 🔴 **MI (Myocardial Infarction)** - Infarto

| Dataset | AUC SR | AUC Baseline | Formula | Complessità |
|---------|--------|--------------|---------|-------------|
| **PTB-XL** | **0.838** | 0.886 | `((age - wav_qrs_peak) + 4.28 - (quality - HR)) * 0.12` | 11 |
| Chapman | 0.820 | 0.896 | `(heart_rate_bpm + age) * 0.18 + 0.5` | 7 |
| Georgia | 0.771 | 0.793 | `(abs(rr_std_ms) - gzip_ratio + n_beats) * 0.14` | 10 |

**🔑 Pattern comune**: `age + heart_rate_bpm` con possibile modulazione wavelet

**Interpretazione**: L'infarto è associato a:
- `age`: forte fattore di rischio
- `wav_wavelet_qrs_peak`: alterazioni morfologiche del QRS
- `heart_rate_bpm`: tachicardia compensatoria

---

## 📈 **STTC (ST/T Changes)** - Alterazioni ST-T

| Dataset | AUC SR | AUC Baseline | Formula | Complessità |
|---------|--------|--------------|---------|-------------|
| **PTB-XL** | **0.825** | 0.875 | `log(sqrt(wav_kurtosis + age + HR + 2.99))` | 9 |
| Chapman | 0.787 | 0.846 | `(heart_rate_bpm + 2.91 + age) * 0.17` | 7 |
| CPSC-2018 | 0.817 | 0.890 | `0.20 * (hjorth_complexity + 2.55 - sex)` | 7 |
| Georgia | 0.735 | 0.831 | `(n_beats + age) * 0.11 + 0.68` | 7 |

**🔑 Pattern comune**: `age + heart_rate_bpm` con features wavelet/complessità

**Interpretazione**: Le alterazioni ST-T sono catturate da:
- `wav_wavelet_coef_kurtosis`: forma anomala dell'onda (code pesanti)
- `age`: degenerazione tissutale
- `hjorth_complexity`: complessità del segnale

---

## 💪 **HYP (Hypertrophy)** - Ipertrofia

| Dataset | AUC SR | AUC Baseline | Formula | Complessità |
|---------|--------|--------------|---------|-------------|
| **Chapman** | **0.959** | 0.918 | `(sqrt(abs(wav_energy_approx)) + kurtosis - (coef_min - abs(-1.17 - age))) * 0.22` | 14 |
| PTB-XL | 0.669 | 0.701 | `wav_wavelet_coef_max * 0.13 + 0.5` | 5 |
| Georgia | 0.573 | 0.642 | `wav_wavelet_coef_min * -0.11 + 0.5` | 5 |

**⚠️ Nota**: Dataset molto sbilanciati (pochi HYP), risultati meno affidabili

**🔑 Pattern comune**: `wav_wavelet_coef_max/min` - ampiezza delle onde

**Interpretazione**: L'ipertrofia causa voltaggio aumentato:
- `wav_wavelet_coef_max`: ampiezza massima (tipicamente alta in LVH)
- `wav_wavelet_coef_min`: profondità onde (inversione in RVH)

---

## ❓ **OTHER** - Altre patologie

| Dataset | AUC SR | AUC Baseline | Formula | Complessità |
|---------|--------|--------------|---------|-------------|
| CPSC-2018 | 0.783 | 0.844 | `0.77 + (hjorth_complexity + age) / 8.87` | 7 |
| Georgia | 0.727 | 0.771 | `log((heart_rate_std - snr_db) * 0.26 + 1.68)` | 8 |
| Chapman | 0.665 | 0.665 | `(square(age) + 5.05 - wav_coef_min) * 0.08` | 8 |
| PTB-XL | 0.584 | 0.509 | `0.70 / (wav_total_energy + 1.56)` | 5 |

**⚠️ Nota**: Categoria eterogenea, difficile da classificare

---

## 📋 Classifica Performance per Patologia

| Patologia | Miglior AUC | Dataset | Formula Candidata |
|-----------|-------------|---------|-------------------|
| **CD** | 0.946 | Chapman | `log(rr_std_ms + heart_rate_bpm + c)` |
| **MI** | 0.838 | PTB-XL | `(age - wav_qrs_peak + c) * k` |
| **STTC** | 0.825 | PTB-XL | `log(sqrt(wav_kurtosis + age + HR))` |
| **HYP** | 0.959* | Chapman | `wav_wavelet_coef_max` (semplificato) |
| **OTHER** | 0.783 | CPSC-2018 | `hjorth_complexity + age` |

*\*AUC alta ma su dataset molto piccolo (14 test samples)*

---

## 🧬 Features Più Importanti (Cross-Dataset)

| Feature | Patologie | Significato Clinico |
|---------|-----------|---------------------|
| `age` | MI, STTC, CD, HYP | Fattore di rischio universale |
| `heart_rate_bpm` | CD, MI, STTC | Risposta fisiologica/patologica |
| `rr_std_ms` | CD, MI | Variabilità ritmica (aritmie) |
| `wav_wavelet_coef_max/min` | HYP | Ampiezza voltaggio (ipertrofia) |
| `wav_wavelet_coef_kurtosis` | STTC | Forma anomala onde |
| `hjorth_complexity` | STTC, OTHER | Complessità segnale |

---

## 🎯 Raccomandazioni per Formula Universale

Per ogni patologia, propongo una **formula candidata semplificata** da validare cross-dataset:

```python
# CD (Conduction Disturbances)
CD_score = log(rr_std_ms + heart_rate_bpm + 3)

# MI (Myocardial Infarction)  
MI_score = (age + heart_rate_bpm) * 0.15

# STTC (ST/T Changes)
STTC_score = log(sqrt(wav_wavelet_coef_kurtosis + age + heart_rate_bpm + 3))

# HYP (Hypertrophy)
HYP_score = wav_wavelet_coef_max * 0.12 + 0.5

# OTHER
OTHER_score = (hjorth_complexity + age) / 10
```

---

## ⚠️ Limitazioni Osservate

1. **SR vs Baseline**: Le formule SR sono generalmente **5-10% sotto** il baseline (Logistic Regression), ma sono **interpretabili**
2. **Dataset sbilanciati**: HYP ha pochissimi campioni in alcuni dataset
3. **Variabilità cross-dataset**: Le formule migliori variano per dataset (popolazioni diverse)
4. **Features mancanti**: Le features `interval_*` (PR, QT, QRS) sono vuote - potrebbero migliorare i risultati

## 🔬 Analisi Cross-Dataset Completata

### **CD (Conduction Disturbances)** - Formula Universale Candidata

| Complessità | Formula | Georgia AUC | Pattern Cross-Dataset |
|-------------|---------|-------------|----------------------|
| **7** | `(n_beats + age) * 0.19 + 0.5` | **0.822** | ✅ Presente in tutti |
| 8 | `log(rr_std_ms + heart_rate_bpm + c)` | 0.802 | Chapman: 0.946 |

**🎯 Formula Universale CD:**
```
CD_score = (n_beats + age) * k + 0.5
```
Dove `k ≈ 0.19`. Semplice, interpretabile, cross-dataset.

---

### **MI (Myocardial Infarction)** - Formula Universale Candidata

| Complessità | Formula | Georgia AUC | Pattern Cross-Dataset |
|-------------|---------|-------------|----------------------|
| **7** | `(n_beats + wav_wavelet_coef_kurtosis) * 0.18 + 0.5` | **0.690** | ✅ |
| 10 | `(abs(rr_std_ms) - gzip_ratio + n_beats) * 0.14` | 0.771 | ✅ |

**🎯 Formula Universale MI:**
```
MI_score = (n_beats + wav_wavelet_coef_kurtosis) * k + 0.5
```
oppure versione più complessa:
```
MI_score = (rr_std_ms + n_beats - comp_sig_gzip_ratio) * k
```

---

### **STTC (ST/T Changes)** - Formula Universale Candidata

| Complessità | Formula | Georgia AUC | Pattern Cross-Dataset |
|-------------|---------|-------------|----------------------|
| **7** | `(n_beats + age) * 0.11 + 0.68` | **0.735** | ✅ Identica a CD! |
| 8 | `log(sqrt(abs(rr_std_ms) + n_beats + c))` | 0.729 | ✅ |

**🎯 Formula Universale STTC:**
```
STTC_score = (n_beats + age) * k + c
```
Molto simile a CD, differisce solo nei coefficienti.

---

### **HYP (Hypertrophy)** - Formula Universale Candidata

| Complessità | Formula | Georgia AUC | Pattern Cross-Dataset |
|-------------|---------|-------------|----------------------|
| **5** | `wav_wavelet_coef_min * -0.11 + 0.5` | **0.573** | ✅ PTB-XL simile |
| 7 | `(wav_coef_min - hjorth_complexity) * -0.12 + 0.5` | 0.561 | ✅ |

**🎯 Formula Universale HYP:**
```
HYP_score = wav_wavelet_coef_min * (-k) + 0.5
```
oppure:
```
HYP_score = (wav_wavelet_coef_min - comp_sig_hjorth_complexity) * (-k) + 0.5
```

---

### **OTHER** - Formula Universale Candidata

| Complessità | Formula | Georgia AUC | Pattern Cross-Dataset |
|-------------|---------|-------------|----------------------|
| **8** | `log((heart_rate_std - snr_db) * 0.26 + 1.68)` | **0.727** | ✅ |
| 5 | `1.41 / (quality_score + 3.14)` | 0.647 | Diverso |

---

## 📊 Riepilogo Formule Universali Proposte

| Patologia | Formula Semplificata | Complessità | AUC Range |
|-----------|---------------------|-------------|-----------|
| **CD** | `(n_beats + age) * 0.19` | 7 | 0.75-0.95 |
| **MI** | `(n_beats + wav_kurtosis) * 0.18` | 7 | 0.69-0.84 |
| **STTC** | `(n_beats + age) * 0.11` | 7 | 0.68-0.83 |
| **HYP** | `wav_coef_min * (-0.11)` | 5 | 0.57-0.96* |
| **OTHER** | `log(heart_rate_std - snr_db)` | 8 | 0.58-0.78 |

*HYP ha alta variabilità per dataset sbilanciati*

---

## 🧬 Pattern Cross-Dataset Più Robusti

### Features Universali (presenti in tutte le patologie):
1. **`n_beats`** - Numero di battiti (proxy per durata/qualità)
2. **`age`** - Fattore di rischio universale

### Features Patologia-Specifiche:
- **CD**: `rr_std_ms` (variabilità ritmo)
- **MI**: `wav_wavelet_coef_kurtosis`, `comp_sig_gzip_ratio`
- **STTC**: `quality_score`, `wav_wavelet_coef_kurtosis`
- **HYP**: `wav_wavelet_coef_min`, `comp_sig_hjorth_complexity`
- **OTHER**: `heart_rate_std`, `snr_db`

---

## 🎯 Osservazione Chiave

**CD e STTC hanno formule quasi identiche!**
```
CD:   (n_beats + age) * 0.19 + 0.5
STTC: (n_beats + age) * 0.11 + 0.68
```

Questo suggerisce che:
1. La struttura della formula è robusta cross-dataset
2. Solo i coefficienti cambiano per patologia
3. Una **formula parametrica** potrebbe funzionare per più patologie

---

### Step 4.3: Stream B - Symbolic Regression for Cardiac Age Prediction

**Script**: [scripts/sr_cardiac_age.py](../scripts/sr_cardiac_age.py)

This module is a core discovery component of the **Kepler-ECG** project, specifically designed for **Cardiac Age Regression**. Its purpose is to evolve transparent, interpretable mathematical formulas that predict a subject's biological age based on ECG features. The primary clinical value lies in the **"Cardiac Age Delta"** (predicted age vs. chronological age): a heart that appears "older" than the patient's actual age can serve as an early biomarker for cardiovascular decline. Utilizing the `PySR` engine, the script searches for the most parsimonious equations that balance predictive accuracy with medical explainability.

#### Input

The script accepts raw feature data and configuration parameters via the `run_sr_cardiac_age` function or CLI:

* **`df` (pd.DataFrame)**: A dataset containing extracted ECG features (morphological, spectral, wavelet) and a target age column.
* **`age_col` (str)**: The name of the target variable (default: `'age'`).
* **`norm_only` (bool)**: If true, the model is trained exclusively on healthy subjects (`NORM`) to establish a baseline for "ideal" cardiac aging.
* **`SRConfig`**: Evolutionary parameters including:
* **`niterations`**: Number of search cycles.
* **`maxsize`**: Maximum complexity of the formula (number of nodes).
* **Operators**: Selection of mathematical building blocks (e.g., `plus`, `mult`, `log`, `exp`, `sqrt`).

#### Output

The module generates a comprehensive discovery package in the specified results directory:

* **`sr_age_results.json`**: A master file containing the "Best Equation," its  score, Mean Absolute Error (MAE), and the parameters used for the search.
* **`age_hall_of_fame.csv`**: The full Pareto front of candidate equations, allowing researchers to choose between a simpler formula or a more complex, accurate one.
* **Validation Metrics**:
* **Comparison vs. Baseline**: Performance relative to standard Linear or Ridge regression.
* **Error Analysis**: Distribution of the "Cardiac Age Delta" across the population.


* **Visualizations**:
* **Scatter Plots**: Predicted vs. Actual Age.
* **Residual Plots**: Highlighting where the model overestimates or underestimates cardiac age.

#### Running

```bash
python scripts/sr_cardiac_age.py --dataset ptb-xl
python scripts/sr_cardiac_age.py --dataset ptb-xl --norm-only
```
---

## 📊 Analisi SR Cardiac Age - Risultati

### Sommario per Dataset

| Dataset | N Samples | R² SR | R² Baseline | MAE | Formula Migliore | Complessità |
|---------|-----------|-------|-------------|-----|------------------|-------------|
| **PTB-XL** | 21,364 | 0.196 | 0.192 | 12.18 | `hjorth + lempel_ziv - wav_max` | 18 |
| **Chapman** | 42,863 | 0.228 | 0.264 | 12.84 | `lempel_ziv - wav_d4 + rr_std - rr_mean` | 13 |
| **CPSC-2018** | 6,724 | 0.100 | 0.175 | 13.82 | `exp(2.06 - wav_coef_max) + 50.6` | 6 |
| **Georgia** | 10,256 | 0.145 | 0.126 | 11.36 | `lzma_ratio + n_beats + rr_std - wav_min` | 17 |

---

### 🔬 Analisi Dettagliata

#### **PTB-XL** (Best R² = 0.196)
```
cardiac_age = abs((22.32 / (heart_rate_std + 0.96)) - 24.14) 
            + ((hjorth_complexity + lempel_ziv + 18.86) - wav_coef_max) * 2.71
```
- SR leggermente migliore del baseline (+0.4% R²)
- Features chiave: `heart_rate_std`, `comp_sig_hjorth_complexity`, `comp_sig_lempel_ziv_complexity`

#### **Chapman** (Best R² = 0.228)
```
cardiac_age = ((lempel_ziv - wav_energy_d4 + rr_std - rr_mean) - wav_coef_std) * 3.79 + 61.22
```
- SR sotto il baseline (-3.6% R²)
- Formula più semplice (complessità 13)
- Features chiave: `comp_sig_lempel_ziv_complexity`, `rr_std_ms`, `rr_mean_ms`

#### **CPSC-2018** (Best R² = 0.100)
```
cardiac_age = exp(2.06 - wav_wavelet_coef_max) + 50.64
```
- **Formula più semplice!** (complessità 6)
- SR sotto il baseline (-7.5% R²)
- Usa solo `wav_wavelet_coef_max`

#### **Georgia** (Best R² = 0.145)
```
cardiac_age = ((lzma_ratio + abs(n_beats + rr_std) - wav_coef_min) * 3.42) 
            + (square(wav_coef_mean_abs - 4.15) + 38.37)
```
- **SR migliore del baseline** (+2% R²)
- Features chiave: `comp_sig_lzma_ratio`, `n_beats`, `rr_std_ms`

---

### 🧬 Pattern Cross-Dataset

#### Features Più Frequenti nelle Formule:

| Feature | PTB-XL | Chapman | CPSC-2018 | Georgia | Totale |
|---------|--------|---------|-----------|---------|--------|
| `comp_sig_lempel_ziv` | ✅ | ✅ | | | 2 |
| `comp_sig_lzma_ratio` | | | | ✅ | 1 |
| `wav_wavelet_coef_max` | ✅ | | ✅ | | 2 |
| `wav_wavelet_coef_min` | | | | ✅ | 1 |
| `wav_wavelet_coef_std` | | ✅ | | | 1 |
| `rr_std_ms` | | ✅ | | ✅ | 2 |
| `heart_rate_std` | ✅ | | | | 1 |
| `hjorth_complexity` | ✅ | | | | 1 |
| `n_beats` | | | | ✅ | 1 |

#### Pattern Comune Identificato:
```
cardiac_age ≈ (compressibility_feature - wavelet_amplitude) * k + offset
```

Dove:
- **compressibility_feature**: `lempel_ziv`, `lzma_ratio`, `hjorth_complexity`
- **wavelet_amplitude**: `wav_coef_max`, `wav_coef_min`, `wav_coef_std`

---

### 📈 Cardiac Age Delta Analysis

| Dataset | Mean Δ | Std Δ | % Older | % Younger | Age Trend Slope |
|---------|--------|-------|---------|-----------|-----------------|
| PTB-XL | +0.46 | 15.14 | 49.9% | 50.1% | **-0.80** |
| Chapman | +0.03 | 15.89 | 45.9% | 54.1% | **-0.78** |
| CPSC-2018 | -0.01 | 16.94 | 45.1% | 54.9% | **-0.90** |
| Georgia | +0.40 | 14.17 | 47.9% | 52.1% | **-0.86** |

**Age Trend Slope ≈ -0.84** (consistente cross-dataset)

Questo significa: per ogni anno di età reale in più, l'età cardiaca predetta aumenta solo di ~0.84 anni. C'è una **regressione verso la media** - i giovani vengono sovrastimati, gli anziani sottostimati.

---

### 🎯 Formula Universale Candidata per Cardiac Age

Basandomi sui pattern comuni, propongo:

```python
# Formula semplice (complessità ~8)
cardiac_age = (comp_sig_lempel_ziv - wav_wavelet_coef_max) * k + mean_age

# Formula estesa (complessità ~12)  
cardiac_age = ((lempel_ziv + rr_std_ms) - wav_coef_max) * k + offset
```

Con `k ≈ 3-4` e `offset ≈ 50-60` (età media della popolazione).

---

### ⚠️ Limitazioni Osservate

1. **R² basso** (0.10-0.26): L'età è difficile da predire solo da features ECG
2. **SR ≤ Baseline** in 2/4 dataset: Le formule semplici non sempre battono Ridge/Linear
3. **Alta variabilità** (MAE ~12-14 anni): Errore medio di ±12 anni
4. **Regressione verso la media**: Slope ~0.84 invece di 1.0

---

### 💡 Interpretazione Clinica

Le features che predicono l'età cardiaca sono legate a:

1. **Complessità del segnale** (`lempel_ziv`, `lzma_ratio`, `hjorth`):
   - Segnali più "complessi" → età maggiore
   - Perdita di regolarità con l'invecchiamento

2. **Ampiezza wavelet** (`wav_coef_max`, `wav_coef_min`):
   - Ampiezze minori → età maggiore
   - Riduzione voltaggio QRS con l'età

3. **Variabilità RR** (`rr_std_ms`):
   - Minore variabilità → età maggiore
   - Perdita di modulazione autonomica

---

🔬 Analisi Cross-Dataset Completa - Cardiac Age

### 📊 Features Più Frequenti per Dataset

| Feature | PTB-XL | Chapman | CPSC-2018 | Georgia | **Totale** |
|---------|--------|---------|-----------|---------|------------|
| `comp_sig_lempel_ziv` | 7 | **17** | - | - | 24 |
| `comp_sig_kolmogorov` | 1 | - | **17** | 3 | 21 |
| `rr_std_ms` | - | **15** | 10 | **15** | 40 |
| `comp_sig_hjorth_complexity` | **11** | 4 | - | - | 15 |
| `comp_sig_bzip2_ratio` | 6 | - | 9 | - | 15 |
| `comp_sig_lzma_ratio` | - | - | - | **11** | 11 |
| `wav_wavelet_coef_max` | 9 | 3 | 1 | - | 13 |
| `wav_wavelet_coef_min` | - | 6 | - | **11** | 17 |
| `wav_wavelet_energy_d4` | 2 | **18** | 9 | 1 | 30 |
| `heart_rate_std` | **9** | - | - | - | 9 |

---

### 🎯 Formule Semplici (C=5-8) - Confronto Cross-Dataset

| Dataset | C | R² | MAE | Formula |
|---------|---|-----|-----|---------|
| **PTB-XL** | 8 | **0.107** | 12.85 | `(wav_energy_d3 - 7.47) / exp(hjorth) + 70` |
| **Chapman** | 7 | **0.130** | 13.82 | `(lempel_ziv + 12.96 - wav_energy_d4) * 4.73` |
| **CPSC-2018** | 6 | **0.100** | 13.82 | `exp(2.06 - wav_coef_max) + 50.6` |
| **Georgia** | 8 | **0.095** | 11.81 | `square(2.90 - qrs_variability) + lzma_ratio + 50.5` |

---

### 🎯 Formule Medie (C=9-13) - Confronto Cross-Dataset

| Dataset | C | R² | MAE | Formula Pattern |
|---------|---|-----|-----|-----------------|
| **PTB-XL** | 12 | **0.137** | 12.58 | `abs(k / (heart_rate_std + c) - c2) + bzip2 + 50` |
| **Chapman** | 13 | **0.228** | 12.84 | `((lempel_ziv - wav_d4 + rr_std - rr_mean) - wav_std) * 3.8 + 61` |
| **CPSC-2018** | 13 | **0.106** | 13.65 | `rr_std + (kolmogorov - qrs_peak*2) * (bzip2 + 5.5) + 59` |
| **Georgia** | 11 | **0.112** | 11.67 | `((rr_std + lzma - qrs_var - coef_mean_abs) * 2.1) + 60` |

---

### 🧬 Pattern Universale Identificato

Analizzando tutte le formule, emerge un **pattern strutturale comune**:

```
cardiac_age = (compressibility - wavelet_amplitude + rr_variability) * k + mean_age
```

Dove:
- **compressibility**: `lempel_ziv`, `kolmogorov`, `lzma_ratio`, `bzip2_ratio`, `hjorth_complexity`
- **wavelet_amplitude**: `wav_coef_max`, `wav_coef_min`, `wav_coef_std`, `wav_energy_d4`
- **rr_variability**: `rr_std_ms`, `rr_mean_ms` (differenza)
- **k** ≈ 2-5
- **mean_age** ≈ 50-61

---

### 📈 Riepilogo Performance

| Dataset | Best R² | Best MAE | Complessità Ottimale |
|---------|---------|----------|---------------------|
| PTB-XL | 0.204 | 12.09 | 20 |
| Chapman | 0.257 | 12.62 | 25 |
| CPSC-2018 | 0.157 | 13.26 | 23 |
| Georgia | 0.154 | 11.31 | 20 |

**Media cross-dataset**: R² ≈ 0.19, MAE ≈ 12.3 anni

---

### 🎯 Formula Universale Candidata

#### **Formula Semplice (C≈7-8)**
```python
cardiac_age = (comp_sig_lempel_ziv - wav_wavelet_coef_max + c1) * k + mean_age
# Esempio: (lempel_ziv - wav_max + 13) * 4.7
```
- R² atteso: 0.08-0.13
- MAE atteso: 12-14 anni

#### **Formula Bilanciata (C≈11-13)**
```python
cardiac_age = ((rr_std_ms + compressibility) - wav_amplitude) * k + mean_age
# Esempio: ((rr_std + lempel_ziv - wav_energy_d4 - wav_coef_max) * 3.5) + 60
```
- R² atteso: 0.11-0.23
- MAE atteso: 11.5-13.5 anni

---

### 💡 Interpretazione Clinica Confermata

Il pattern cross-dataset conferma che l'**età cardiaca** è predetta da:

1. **↑ Compressibilità** (segnale più "regolare", meno informazione) → **più vecchio**
   - `lempel_ziv`, `kolmogorov`, `lzma_ratio` aumentano con età

2. **↓ Ampiezza wavelet** (voltaggio ridotto) → **più vecchio**
   - `wav_coef_max`, `wav_energy_d4` diminuiscono con età

3. **↓ Variabilità RR** (perdita modulazione autonomica) → **più vecchio**
   - `rr_std_ms` diminuisce con età

**Formula interpretabile finale:**
```
età_cardiaca ≈ 60 + 3.5 × (complessità_segnale - ampiezza_onde + variabilità_RR_ridotta)
```

---

### Step 4.4: Multi-Dataset Wave Delineation Pipeline

**Script**: [scripts/extract_waves.py](../scripts/extract_waves.py)

The **Multi-Dataset Wave Delineation Pipeline** is a robust signal processing engine designed to extract precise PQRST fiducial points from various ECG databases (PTB-XL, Chapman, CPSC, etc.). By leveraging the `NeuroKit2` library and `wfdb` format support, it identifies the onset, peak, and offset of cardiac waves. This module is foundational for **Phase 4.5**, as it converts raw voltage-time signals into clinical interval metrics (RR, PR, QRS, QT) and automatically calculates standard heart-rate corrected intervals (QTc) using multiple clinical formulas.

#### Input

The script is designed for high-volume batch processing and accepts:

* **`--dataset` (str)**: The registry name of the dataset to process (e.g., `'ptb-xl'`, `'chapman'`).
* **`--input` (Path)**: An optional direct path to a directory containing ECG records (overrides the registry).
* **`--sampling_rate` (int, default: 500)**: The frequency of the input signal to ensure accurate time-series analysis.
* **`--lead` (int, default: 0)**: The lead index (e.g., Lead I or Lead II) to be used as the primary source for delineation.
* **`--max_records` (Optional)**: A limit for the number of files to process, ideal for debugging or sampling.

#### Output

The pipeline produces a comprehensive results package in the specified output directory:

* **`wave_features.csv`**: A dense feature matrix containing:
* **Fiducial Points**: Sample indices for P-peak, QRS-onset, R-peak, T-offset, etc.
* **Calculated Intervals**: RR, PR, QRS, and QT durations in milliseconds.
* **Baseline QTc Values**: Pre-calculated Bazett, Fridericia, Framingham, and Hodges corrections.


* **`delineation_summary.json`**: A master metadata file reporting:
* **Processing Stats**: Total records processed vs. successful delineations.
* **Population Statistics**: Mean and standard deviation for all extracted intervals and heart rates.
* **Success Rate**: A percentage-based reliability score for the dataset quality.


* **Console Telemetry**: A real-time progress bar (tqdm) and final summary table highlighting clinical averages.

#### Running

```bash
python scripts/extract_waves.py --dataset ptb-xl
```
---

### Step 4.5: QTc Dataset Preparation Utility

**Script**: [scripts/prepare_qtc_dataset.py](../scripts/prepare_qtc_dataset.py)

The **QTc Dataset Preparation Module** is a robust data engineering script designed to transform raw wave features into high-quality datasets for medical formula discovery. It implements a multi-stage filtering pipeline that enforces clinical quality standards (e.g., valid QT and RR ranges) and integrates diagnostic metadata to categorize patients. A key feature of this module is the calculation of a "Ground Truth" QTc reference—an idealized QT interval corrected to a heart rate of 60 bpm—using population-level regression. This ensures the resulting data is perfectly formatted for training **Symbolic Regression** models to discover next-generation heart rate correction formulas.

#### Input

The script can be executed via CLI or integrated as a module, requiring:

* **`--input` (Path)**: The CSV file generated by the wave extraction pipeline (e.g., `wave_features.csv`), containing raw millisecond measurements of PQRST landmarks.
* **`--dataset` (str)**: The name of the registry dataset (e.g., `'ptb-xl'`, `'chapman'`) to automatically locate metadata and labels.
* **Quality Thresholds**: Internally defined ranges for clinical physiological limits (e.g., heart rate between 30 and 200 BPM, QT intervals between 200ms and 800ms).

#### Output

The module produces a refined "SR-Ready" (Symbolic Regression Ready) package:

* **`qtc_sr_dataset_all.csv`**: The complete cleaned dataset including original intervals, standard corrections (Bazett, Fridericia), and the calculated reference.
* **`qtc_sr_dataset_norm.csv`**: A specialized subset containing only "Healthy" (NORM) subjects, ideal for learning baseline cardiac physiology.
* **`qtc_prep_report.json`**: A detailed analytical summary featuring:
* **Filter Statistics**: A breakdown of how many records were dropped due to noise or physiological outliers.
* **Correlation Analysis**: Pearson 'r' values showing how much traditional formulas still depend on heart rate.
* **Dataset Sizes**: Final counts for training and validation cohorts.


* **Console Logs**: Real-time feedback on the normalization process and the strength of the HR-QT correlation in the processed batch.

#### Running

```bash
python scripts/prepare_qtc_dataset.py --dataset ptb-xl
```
---

### Step 4.6: Stream C - QTc Formula Discovery via Symbolic Regression

**Script**: [scripts/sr_qtc_discovery.py](../scripts/sr_qtc_discovery.py)

The **QTc Discovery Module** is an advanced scientific tool within Phase 4.5 of the Kepler-ECG project. It leverages **Symbolic Regression (via PySR)** to discover interpretable mathematical formulas for heart rate-corrected QT intervals (QTc). The script moves beyond standard clinical rules (like Bazett or Fridericia) by exploring three mathematical architectures: **Direct prediction** (formula maps raw features to QTc), **Factor-based** (QT multiplied by a correction function), and **Additive** (QT plus a correction term). The primary objective is to evolve a formula that minimizes the correlation between the corrected QT and heart rate, providing a more stable diagnostic metric for different physiological states.

#### Input

The script is designed for automated execution using pre-prepared datasets:

* **`--dataset` (str)**: The name of the registered dataset (e.g., `'ptb-xl'`) to automatically fetch the prepared CSV.
* **`--input` (Path)**: Alternatively, a direct path to an SR-ready CSV (e.g., `qtc_sr_dataset_all.csv`) containing cleaned QT and RR intervals.
* **`--approach` (str, default: 'all')**: Choice of mathematical structure (`direct`, `factor`, or `additive`).
* **`--iterations` (int, default: 150)**: The depth of the evolutionary search.
* **`--feature_set` (str)**: Defines which physiological variables to include (e.g., `rr_only` or `extended` including age/gender).

#### Output

The module generates a comprehensive research package in the results folder:

* **`task4_sr_report.json`**: A master file containing the "Best Overall" equation, its mathematical complexity, and the achieved loss.
* **`sr_results_[approach].csv`**: The full "Hall of Fame" (Pareto front) for each approach, listing candidate equations from simplest to most accurate.
* **Comparison Metrics**:
* **|r(HR)|**: The absolute Pearson correlation between the new QTc and Heart Rate (lower is better).
* **Improvement Factor**: A quantitative measure of how much better the new formula performs compared to the Bazett baseline.


* **Visualizations**:
* **Pareto Front Plot**: Shows the trade-off between equation complexity and error.
* **Correlation Charts**: Visualizes the decoupling of QT from Heart Rate using the new formula.

#### Running

```bash
python scripts/sr_qtc_discovery.py --dataset ptb-xl --approach all
```
---

### Step 4.7: QTc Formula Validation & Clinical Benchmarking

**Script**: [scripts/validate_qtc_formulas.py](../scripts/validate_qtc_formulas.py)

The **QTc Formula Validation** module is a scientific verification utility within the Kepler-ECG pipeline. Its primary purpose is to rigorously assess the "Heart Rate Independence" of discovered mathematical formulas compared to traditional clinical standards (Bazett, Fridericia, Framingham, and Hodges). It evaluates how well a formula "decouples" the QT interval from the heart rate across diverse physiological ranges (Bradycardia, Normal, and Tachycardia). This script provides the statistical proof required to demonstrate that the new Kepler formulas offer superior stability and more accurate diagnostic thresholds for identifying patients at risk of Arrhythmia.

#### Input

The script operates on cleaned, high-resolution feature data:

* **`--dataset` (str)**: The name of the registered dataset (e.g., `'ptb-xl'`) to automatically fetch the SR-ready CSV.
* **`--input` (Path)**: Alternatively, a direct path to a CSV (e.g., `qtc_sr_dataset_all.csv`) containing QT intervals, RR intervals, and diagnostic labels.
* **Predefined Kepler Formulas**: The script internally defines the mathematical expressions for discovered models like "Kepler-Cubic" or "Kepler-Linear" to test them against the population.

#### Output

The module generates a comprehensive validation package in the `validation/` directory:

* **`qtc_validation_report.json`**: A master file containing:
* **HR Independence Ranking**: All formulas ranked by the absolute Pearson correlation () with heart rate (lower is better).
* **Bin Analysis**: Statistical stability of each formula across specific BPM ranges.
* **Clinical Threshold Analysis**: Percentage of patients classified as "Prolonged QT" (>450ms) or "High Risk" (>500ms) by each formula.


* **`validation_summary.png`**: A high-resolution plot showing the regression of QTc vs. Heart Rate for all competing models.
* **`hr_bin_comparison.png`**: A box-plot visualization showing the distribution of corrected intervals across different heart rate categories.
* **Console Leaderboard**: A final summary highlighting the "Improvement Factor" of the best Kepler formula over the Bazett baseline.

```bash
python scripts/validate_qtc_formulas.py --dataset ptb-xl
```
---

### Step 4.8: QTc Discovery Report Generator

**Script**: [scripts/generate_qtc_report.py](../scripts/generate_qtc_report.py)

## 📑 Kepler-ECG: QTc Discovery Report Generator

The **QTc Discovery Report Generator** is the final documentation engine for the **Phase 4.5** research stream. It serves as a scientific synthesizer, automatically gathering technical outputs from wave delineation, symbolic regression, and clinical validation tasks to produce a human-readable research summary. The script generates multiple formats (JSON, Markdown, and Clinical Interpretation) to facilitate both further automated processing and peer-reviewed publication. It specifically focuses on quantifying the "Improvement Factor" of the discovered Kepler formulas over century-old standards like Bazett's formula.

#### Input

The script functions as an aggregator, looking for JSON artifacts generated in previous pipeline steps:

* **`--validation` (Path)**: The validation JSON (from Task 5) containing HR-independence rankings and clinical threshold data.
* **`--sr` (Path)**: The Symbolic Regression report (from Task 4) detailing the best-discovered equations and their mathematical complexity.
* **`--waves` (Path)**: The wave delineation summary (from Task 1) providing population statistics and processing success rates.
* **`--dataset` (str)**: Used to automatically resolve file paths if the standard project structure is maintained.

#### Output

The module produces a suite of "Publication-Ready" documents:

* **`qtc_final_report.json`**: A master data file containing every metric, formula, and performance benchmark from the stream.
* **`qtc_final_report.md`**: A formatted Markdown report suitable for GitHub documentation or laboratory notebooks.
* **`clinical_interpretation.md`**: A medically-focused document explaining the rationale behind the new formulas and their potential impact on misdiagnosis rates in Tachycardia/Bradycardia.
* **Executive Summary**: A console-based report highlighting the "Success Status" (Pass/Fail) based on the project's predefined scientific targets.

```bash
python scripts/generate_qtc_report.py --dataset ptb-xl
```
---